In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

%matplotlib inline

### Carga de datos de entrenamiento y test

In [2]:
df = pd.read_csv('data/clean/train_clean_v2.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')
df.shape, df_targets.shape

((10795392, 27), (10795392, 24))

In [3]:
df_test = pd.read_csv('data/test_ver2.csv', dtype={'sexo': str, 'age': str, 'ind_nuevo': str,
                                                    'indrel_1mes': str, 'antiguedad': str, 'ult_fec_cli_lt': str,
                                                    'indext': str, 'conyuemp': str}, parse_dates=['fecha_dato', 'fecha_alta'])
df_test.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [4]:
df_copy = df.copy()

### Preprocesado de los datos de test
En esta sección se le hace un preprocesado a los datos de test que consiste en:
* Eliminar columnas que tienen muchos datos vacíos
* Cambiar el tipo de datos de algunas columnas
* Rellenar los datos faltantes en ciertas filas, **no se pueden eliminar filas porque en la _submission_ de Kaggle se requieren de la totalidad de registros del dataset**
* Cambiar las variables categóricas a numéricas
* **Se guarda el dataset de test**

In [5]:
df_test.shape

(929615, 24)

In [6]:
#Eliminar columnas
df_test.drop(labels=['conyuemp', 'ult_fec_cli_1t'], inplace=True, axis=1)

In [12]:
#Eliminar filas
#missing = df_test[df_test.isnull().any(axis=1)]
#missing_index = missing.index
#df_test.drop(missing_index, inplace=True, axis=0)

In [7]:
#Cambio de tipo de variable
df_test.age = df_test.age.astype('int64')
df_test.ind_nuevo = df_test.ind_nuevo.astype('int64')
df_test.antiguedad = df_test.antiguedad.astype('int64')
df_test.indrel_1mes = df_test.indrel_1mes.astype('float64')

In [8]:
df_test.renta.replace('         NA', 0, inplace=True)

In [9]:
df_test.renta = df_test.renta.astype('float64')

In [10]:
df_test.indrel_1mes.replace(np.nan, 1, inplace=True)

In [11]:
df_test.sexo.replace(np.nan, 0, inplace=True)
df_test.tiprel_1mes.replace(np.nan, 0, inplace=True)
df_test.canal_entrada.replace(np.nan, 0, inplace=True)
df_test.cod_prov.replace(np.nan, 0, inplace=True)
df_test.nomprov.replace(np.nan, 0, inplace=True)
df_test.segmento.replace(np.nan, 0, inplace=True)

In [13]:
#Cambio de formato de fecha
df_test['fecha_dato_year'] = df_test.fecha_dato.dt.year
df_test['fecha_dato_month'] = df_test.fecha_dato.dt.month
df_test['fecha_dato_day'] = df_test.fecha_dato.dt.day

df_test['fecha_alta_year'] = df_test.fecha_alta.dt.year
df_test['fecha_alta_month'] = df_test.fecha_alta.dt.month
df_test['fecha_alta_day'] = df_test.fecha_alta.dt.day

In [15]:
%%time
#Cambio de atributos categóricos a numéricos
cols = df_test.select_dtypes(['object']).columns

for col in cols:
    print(col, '¡LISTO!')
    attribute_vals = df_test[col].value_counts().index
    attribute_counts = np.arange(1, len(attribute_vals) + 1)
    df_test.replace(attribute_vals, attribute_counts, inplace=True)

ind_empleado ¡LISTO!
pais_residencia ¡LISTO!
sexo ¡LISTO!
tiprel_1mes ¡LISTO!
indresi ¡LISTO!
indext ¡LISTO!
canal_entrada ¡LISTO!
indfall ¡LISTO!
nomprov ¡LISTO!
segmento ¡LISTO!
CPU times: user 14min 13s, sys: 7.31 s, total: 14min 20s
Wall time: 14min 18s


In [16]:
cols = df_test.columns.tolist()
cols_ordered = cols[0:1] + cols[22:25] + cols[1:7] + cols[25:] + cols[7:22]

In [17]:
df_test = df_test[cols_ordered]

In [18]:
%%time
#Guardar
df_test.to_csv('data/clean/test_clean.csv', index=False)

CPU times: user 48.3 s, sys: 64 ms, total: 48.4 s
Wall time: 49.8 s


---

### Entrenamiento y testeo
En esta sección se hace el entrenamiento de los datos y se testea. Por ahora se está usando el _RandomForestClassifier_

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
df_test = pd.read_csv('data/clean/test_clean.csv')
df_test.shape

(929615, 28)

Función de entrenamiento, testeo y generación de archivo de _submission_

In [7]:
def results(x_train, y_train, x_test, target_cols, out_file, result_file, df_result, ncodpers_last_month):
    
    #x_train: array de entrenamiento - features
    #y_train: array de entrenamiento - targets
    #targets_cols: lista de los nombres de los targets
    #out_file: nombre del archivo txt de salida 
    #result_file: nombre del archivo csv de salida
    #df_result: dataframe para generar el archivo de salida
    #ncodpers_last_month: serie de ncodpers que están en el mes anterior del test
    
    s = time()
    for i, col in enumerate(target_cols):
        f = open('results/txt/' + out_file + '.txt', 'a')
        start = time()
        rf = RandomForestClassifier(n_jobs=4)
        rf.fit(x_train, y_train[:, i])
        preds = list(rf.predict(x_test))
        df_result[col] = preds
        end = time()
        f.write("{} {} {:0.4f} min\n".format(i, col, (end - start)/float(60)))
        f.close()

    rs = df_result.columns.tolist()[1:]

    for i in range(len(df_result)):
        
        ncodper = df_result.iloc[i]['ncodpers'] #Código individual
        
        if ncodper in ncodpers_last_month.values:
            pre_targ = y_train[ncodpers_last_month[ncodpers_last_month == ncodper].index]
            post_targ = df_result.iloc[i, 1:-1].as_matrix()
            line = (pre_targ - post_targ).reshape((24))
            df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == -1])
        else:
            line = df_result.iloc[i, 1:-1].as_matrix()
            df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
            
    e = time()
    f = open('results/txt/' + out_file + '.txt', 'a')
    f.write("\nTiempo total: {:0.4f} min".format((e-s)/float(60)))
    f.close()
    df_result[['ncodpers', 'added_products']].to_csv('results/submissions/' + result_file + '.csv', index=False)

Datos de entrenamiento

In [8]:
#Dataframe que guarda el resultado del testeo para generar el archivo de sumisión
resultados = pd.DataFrame(columns= ['ncodpers'] + df_targets.columns.tolist() + ['added_products'])
resultados['ncodpers'] = df_test['ncodpers']

ncodpers_last_month = df_copy[df_copy.fecha_dato == df_copy.fecha_dato.value_counts().sort_index().index[-1]]['ncodpers']
df_copy.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1, inplace=True)

x = df_copy.as_matrix()
y = df_targets.as_matrix()

df_test.drop(['fecha_dato', 'fecha_alta', 'ncodpers', 'ind_nuevo'], axis=1, inplace=True)
x_test = df_test.as_matrix()

In [9]:
columns = df_targets.columns.tolist()

## Resultados

#### Resultados del primer intento
Sólo se usó RandomForestClassifier

In [96]:
res = pd.read_csv('resultados_notebook.csv')
res.shape

(929615, 26)

In [97]:
submission = res[['ncodpers', 'added_products']]

In [98]:
%%time
submission.to_csv('submission.csv', index=False)

CPU times: user 3.01 s, sys: 48 ms, total: 3.06 s
Wall time: 3.35 s


### Resultados con diferente preprocesamiento de datos
En esta sección se muestra el testeo de datos con diferentes procesamientos de datos

#### PCA
Se hizo con un PCA de 10 componentes iniciando. El resto se está ejecutando a través del script correspondiente (`scripts/submissions.py`)

In [12]:
from sklearn.decomposition import PCA

In [27]:
%%time
PCA = PCA(n_components=10)
x_train_pca = PCA.fit_transform(x)
x_test_pca = PCA.fit_transform(x_test)

CPU times: user 1min, sys: 2.81 s, total: 1min 2s
Wall time: 24.9 s


In [ ]:
results(x_train=x_train_pca,
        y_train=y,
        x_test=x_test_pca,
        target_cols=columns,
        out_file='out_pca_n10',
        result_file='resultados_pca_n10',
        df_result=resultados)

In [44]:
res = pd.read_csv('scripts/results/csv/resultados_pca_n10.csv')
res.shape

(929615, 26)

In [45]:
res[['ncodpers', 'added_products']].to_csv('submissions/pca_n10.csv', index=False)